In [54]:
import pandas as pd
from nltk.tokenize import word_tokenize
from string import punctuation
from nltk.corpus import stopwords
from nltk.stem import SnowballStemmer
nltk.download('vader_lexicon')
from nltk.sentiment.vader import SentimentIntensityAnalyzer
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.linear_model import LogisticRegression

[nltk_data] Downloading package vader_lexicon to /root/nltk_data...
[nltk_data]   Package vader_lexicon is already up-to-date!


In [55]:
data = pd.read_csv("amazon_reviews.txt", delimiter = "\t")

In [59]:
data.head()

,DOC_ID,LABEL,RATING,VERIFIED_PURCHASE,PRODUCT_CATEGORY,PRODUCT_ID,PRODUCT_TITLE,REVIEW_TITLE,REVIEW_TEXT
0,1,__label1__,1,N,PC,B00008NG7N,"Targus PAUK10U Ultra Mini USB Keypad, Black",useful,"When least you think so, this product will sav..."
1,2,__label1__,1,Y,Wireless,B00LH0Y3NM,Note 3 Battery : Stalion Strength Replacement ...,New era for batteries,Lithium batteries are something new introduced...
2,3,__label1__,0,N,Baby,B000I5UZ1Q,"Fisher-Price Papasan Cradle Swing, Starlight",doesn't swing very well.,I purchased this swing for my baby. She is 6 m...
3,4,__label1__,1,N,Office Products,B003822IRA,Casio MS-80B Standard Function Desktop Calculator,Great computing!,I was looking for an inexpensive desk calcolat...
4,5,__label1__,1,N,Beauty,B00PWSAXAM,Shine Whitening - Zero Peroxide Teeth Whitenin...,Only use twice a week,I only use it twice a week and the results are...


In [56]:
#  For positive it is 1 and for negative it is 0
data.RATING = [1 if i>3  else 0 for i in data.RATING]

In [57]:
# Tokenizing all the words
tok=word_tokenize(reviews_combined.lower())

#lets remove some unnecessary terms
punct_stops=list(punctuation)+list(stopwords.words('english'))
# Snowball stemmer
snow=SnowballStemmer('english')
# vader sentiment analyser
analyser=SentimentIntensityAnalyzer()
# Logistic Regression
logreg=LogisticRegression()

In [58]:
def clean_text(sent):
    tokens=word_tokenize(sent.lower())
    stop_nltk=stopwords.words('english')
    punct_stops=list(punctuation)+list(stop_nltk)
    stemmed=[snow.stem(i) for i in tokens
             if i not in punct_stops and len(i)>2 ]
    res=" ".join(stemmed)
    return res

In [49]:
data['cl']=data.REVIEW_TEXT.apply(clean_text)

In [50]:
def getvader_sentiment(sent):
    return analyser.polarity_scores(sent)['compound']

In [51]:
sentpred=data['REVIEW_TEXT'].apply(getvader_sentiment)

In [52]:
data['sentpred']=sentpred.apply(lambda i:1 if i>0.5 else 0)

In [53]:
accuracy_score(data.RATING,data.sentpred)

0.785952380952381

In [31]:
x=data.REVIEW_TEXT.values
y=data.RATING.values

In [32]:
xtrain,xtest,ytrain,ytest=train_test_split(x,y,test_size=0.3,random_state=42)

In [33]:
vectorizer=CountVectorizer(max_features=2500)

In [34]:
#Extract the features and transform, here transform means frequency count again
xtrainbow=vectorizer.fit_transform(xtrain)

In [35]:
#only transform -- get the count of terms in every document, the test data
xtestbow=vectorizer.transform(xtest)

In [36]:
xtestbow.shape,xtrainbow.shape

((6300, 2500), (14700, 2500))

In [38]:
logreg.fit(xtrainbow,ytrain)

/usr/local/lib/python3.7/dist-packages/sklearn/linear_model/_logistic.py:940: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)


LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=100,
                   multi_class='auto', n_jobs=None, penalty='l2',
                   random_state=None, solver='lbfgs', tol=0.0001, verbose=0,
                   warm_start=False)

In [60]:
ytest_pred=logreg.predict(xtestbow)

accuracy_score(ytest,ytest_pred)

0.8415873015873016